In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
import warnings
import time

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# --- CONFIGURATION ---
FILE_PATH = r'C:\Users\yashs\Desktop\Project\Fintech Credit Risk\accepted_2007_to_2018Q4.csv'
TARGET_YEARS = [2016, 2017, 2018]
CHUNK_SIZE = 100000
TEST_SIZE = 0.2
RANDOM_STATE = 42

# XGBoost Hyperparameters (tuned for recall/profit)
XGB_PARAMS = {
    'n_estimators': 100,
    'learning_rate': 0.1,
    'max_depth': 5,
    'scale_pos_weight': 4,  # Critical: Penalty for missing defaults (4:1 cost ratio)
    'n_jobs': -1,
    'use_label_encoder': False,
    'eval_metric': 'logloss'
}

def load_modern_data(file_path, years):
    """
    Reads the large CSV in chunks to filter for specific years without blowing up RAM.
    """
    print(f"--> Starting Data Extraction for years: {years}...")
    start_time = time.time()
    
    chunks = []
    
    # Iterate through file
    for chunk in pd.read_csv(file_path, chunksize=CHUNK_SIZE, low_memory=False):
        
        # 1. Basic filter: Closed loans only
        chunk = chunk[chunk['loan_status'].isin(['Fully Paid', 'Charged Off'])]
        
        # 2. Year extraction (Handling 'Mon-YYYY' format)
        chunk['year'] = chunk['issue_d'].astype(str).str[-4:].astype(int)
        
        # 3. Filter for target era
        valid_rows = chunk[chunk['year'].isin(years)]
        
        if not valid_rows.empty:
            chunks.append(valid_rows)
            # Optional: Print progress every few chunks if needed
            # print(f"   Buffered {len(valid_rows)} rows...")

    df = pd.concat(chunks, axis=0)
    elapsed = time.time() - start_time
    
    print(f"--> Extraction Complete. Loaded {len(df):,} rows in {elapsed:.1f}s.")
    return df

def feature_engineering(df):
    """
    Cleans raw data and generates financial metrics.
    """
    print("--> Processing Features & Engineering Targets...")
    
    # 1. Financial Metric: Actual Profit (Cash In - Cash Out)
    df['Actual_Profit'] = df['total_pymnt'] - df['funded_amnt']
    
    # 2. Target Definition: 1 = Default (Charged Off), 0 = Paid
    df['target'] = df['loan_status'].apply(lambda x: 1 if x == 'Charged Off' else 0)
    
    # 3. Feature Selection
    features = [
        'loan_amnt', 'term', 'int_rate', 'sub_grade', 'emp_length',
        'home_ownership', 'annual_inc', 'verification_status',
        'purpose', 'dti', 'fico_range_high', 'revol_util'
    ]
    
    X = df[features].copy()
    y = df['target']
    
    # 4. Cleaning
    # Term: " 36 months" -> 36
    X['term'] = X['term'].astype(str).str.replace(' months', '').astype(int)
    
    # Emp Length: Map text to int, fill NaN with 0
    emp_map = {
        '< 1 year': 0, '1 year': 1, '2 years': 2, '3 years': 3, '4 years': 4,
        '5 years': 5, '6 years': 6, '7 years': 7, '8 years': 8, '9 years': 9, '10+ years': 10
    }
    X['emp_length'] = X['emp_length'].map(emp_map).fillna(0).astype(int)
    
    # Numeric Imputation (Median is safer for income/dti)
    for col in ['dti', 'revol_util', 'annual_inc']:
        X[col] = X[col].fillna(X[col].median())
        
    # 5. Encoding
    # Ordinal Encode Sub-Grade (A1->0, A2->1, ... G5->34)
    # This preserves the "Risk Ranking" inherent in grades
    grade_map = {grade: i for i, grade in enumerate(sorted(X['sub_grade'].unique()))}
    X['sub_grade'] = X['sub_grade'].map(grade_map)
    
    # One-Hot Encode others
    X = pd.get_dummies(X, columns=['home_ownership', 'verification_status', 'purpose'], drop_first=True)
    
    print(f"--> Data Ready. Feature Matrix Shape: {X.shape}")
    print(f"--> Default Rate: {y.mean():.2%}")
    
    return X, y, df[['Actual_Profit', 'sub_grade', 'int_rate', 'term']]

def generate_visuals(results, model, feature_names):
    """
    Generates the 3 key portfolio charts.
    """
    sns.set_style("whitegrid")
    plt.rcParams['font.family'] = 'sans-serif'
    
    # Chart 1: The Money Slide (Profit Distribution)
    plt.figure(figsize=(12, 6))
    results['Outcome'] = results.apply(lambda x: 'REJECTED (Risk Avoided)' if x['Pred'] == 1 else 'APPROVED (Profit Generated)', axis=1)
    
    sns.histplot(data=results, x='Actual_Profit', hue='Outcome', element="step", bins=50, 
                 palette={'REJECTED (Risk Avoided)': '#FF4B4B', 'APPROVED (Profit Generated)': '#2ECC71'}, alpha=0.6)
    
    plt.axvline(0, color='black', linestyle='--', linewidth=1)
    plt.title('Financial Impact: Identifying Toxic Assets', fontsize=16, fontweight='bold', pad=15)
    plt.xlabel('Net Profit/Loss per Loan (€)')
    plt.legend(title='Model Decision', labels=['APPROVED (Profit Generated)', 'REJECTED (Risk Avoided)'])
    plt.tight_layout()
    plt.show()
    
    # Chart 2: Feature Importance
    plt.figure(figsize=(10, 6))
    importances = pd.Series(model.feature_importances_, index=feature_names).sort_values(ascending=False).head(10)
    sns.barplot(x=importances.values, y=importances.index, palette='viridis')
    plt.title('Top 10 Drivers of Credit Risk', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

    # Chart 3: Risk by Grade (Validation)
    plt.figure(figsize=(12, 6))
    grade_risk = results.groupby('sub_grade')['target'].mean().reset_index().sort_values('sub_grade')
    
    sns.barplot(x='sub_grade', y='target', data=grade_risk, palette='magma', order=grade_risk['sub_grade'])
    plt.axhline(results['target'].mean(), color='red', linestyle='--', label='Portfolio Avg')
    plt.title('Default Rate by Loan Sub-Grade', fontsize=16, fontweight='bold')
    plt.ylabel('Actual Default Rate')
    plt.legend()
    plt.tight_layout()
    plt.show()

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    
    # 1. Load Data
    raw_df = load_modern_data(FILE_PATH, TARGET_YEARS)
    
    # 2. Preprocessing
    X, y, financial_df = feature_engineering(raw_df)
    
    # 3. Split (Stratified to keep default rate consistent)
    # We pass financial_df to keep the profit data aligned with the split
    X_train, X_test, y_train, y_test, df_train, df_test = train_test_split(
        X, y, financial_df, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
    )
    
    # 4. Train Model
    print(f"--> Training XGBoost on {len(X_train):,} loans...")
    model = xgb.XGBClassifier(**XGB_PARAMS)
    model.fit(X_train, y_train)
    print("--> Training Complete.")
    
    # 5. Financial Evaluation
    print("--> Calculating ROI on Test Set...")
    df_test['Pred'] = model.predict(X_test)
    df_test['target'] = y_test  # Add target back for visualization
    
    baseline_profit = df_test['Actual_Profit'].sum()
    model_profit = df_test[df_test['Pred'] == 0]['Actual_Profit'].sum()
    value_created = model_profit - baseline_profit
    
    print("\n" + "="*40)
    print(f" RESULTS SUMMARY (Test Set: {len(X_test):,} loans)")
    print("="*40)
    print(f"Baseline Profit (No Model):   €{baseline_profit:,.0f}")
    print(f"Model Profit (Optimization):  €{model_profit:,.0f}")
    print(f"NET VALUE CREATED:            €{value_created:,.0f}")
    print("="*40 + "\n")
    
    # 6. Generate Charts
    generate_visuals(df_test, model, X.columns)